In [1]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
import numpy as np
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
from sklearn.metrics import mean_absolute_error
from typing import List, Dict
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_autogluon
importlib.reload(preprocesamiento)
importlib.reload(model_autogluon)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

c:\Users\Usuario\.conda\envs\py311lab3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


# Experimento 6: 
- Autogluon: Error-Driven Ensemble Weighting
- NO FUNCIONO
- Kaggle =  


In [ ]:
# class ErrorDrivenEnsemble:
#     def __init__(self, val_window_size: int = 24):
#         """
#         Inicializa el ensamblador con el tamaño de la ventana de validación
#         :param val_window_size: Número de períodos a usar para validación (por defecto 24 meses)
#         """
#         self.val_window_size = val_window_size
#         self.predictors = []
#         self.weights = []
#         self.best_combination = None

#     def add_predictor(self, predictor_config: Dict):
#         """
#         Agrega una configuración de predictor al ensamblaje
#         :param predictor_config: Dict con configuración. Ejemplo:
#             {
#                 'name': 'deepar_12w',
#                 'train_kwargs': {
#                     'num_val_windows': 12,
#                     'hyperparameters': {'DeepAR': {...}}
#                 }
#             }
#         """
#         self.predictors.append(predictor_config)

#     def _prepare_validation_data(self, data: TimeSeriesDataFrame):
#         """Prepara los datos de entrenamiento y validación"""
#         train_data = data.iloc[:-self.val_window_size]
#         val_data = data.iloc[-self.val_window_size:]
#         return train_data, val_data

#     def _train_predictor(self, train_data: TimeSeriesDataFrame, config: Dict):
#         """Entrena un predictor individual"""
#         predictor = TimeSeriesPredictor(
#             target='target',
#             prediction_length=2,
#             freq="M",
#             eval_metric="MAPE",
#             quantile_levels=[0.1, 0.5, 0.9]
#         )
        
#         predictor.fit(
#             train_data=train_data,
#             **config['train_kwargs']
#         )
#         return predictor

#     def _evaluate_predictor(self, predictor, val_data: TimeSeriesDataFrame):
#         """Evalúa un predictor en los datos de validación"""
#         preds = predictor.predict(val_data)
#         val_targets = val_data['target']
        
#         # Alinear índices para comparación
#         aligned_preds = preds.reset_index().merge(
#             val_data.reset_index()[['item_id', 'timestamp']],
#             on=['item_id', 'timestamp']
#         ).set_index(['item_id', 'timestamp'])
        
#         mae = mean_absolute_error(val_targets, aligned_preds['mean'])
#         return mae, preds

#     def optimize_weights(self, data: TimeSeriesDataFrame):
#         """
#         Entrena y evalúa todos los predictores, calculando los pesos óptimos
#         :param data: Datos completos (TimeSeriesDataFrame)
#         :return: Dict con resultados de optimización
#         """
#         train_data, val_data = self._prepare_validation_data(data)
#         results = []
        
#         # Entrenar y evaluar cada predictor
#         for config in self.predictors:
#             print(f"\nEntrenando {config['name']}...")
#             try:
#                 predictor = self._train_predictor(train_data, config)
#                 mae, preds = self._evaluate_predictor(predictor, val_data)
                
#                 results.append({
#                     'name': config['name'],
#                     'predictor': predictor,
#                     'mae': mae,
#                     'val_predictions': preds
#                 })
#                 print(f"{config['name']} - MAE: {mae:.2f}")
                
#             except Exception as e:
#                 print(f"Error entrenando {config['name']}: {str(e)}")
#                 continue
        
#         if not results:
#             raise ValueError("Ningún predictor se entrenó exitosamente")
        
#         # Calcular pesos (inversamente proporcional al error)
#         maes = np.array([r['mae'] for r in results])
#         self.weights = (1 / maes) / (1 / maes).sum()
        
#         # Guardar resultados
#         self.results = results
#         self.best_combination = {
#             'names': [r['name'] for r in results],
#             'weights': self.weights.tolist(),
#             'mean_mae': maes.mean(),
#             'best_mae': maes.min()
#         }
        
#         return self.best_combination

#     def predict(self, data: TimeSeriesDataFrame):
#         """
#         Genera predicciones ensambladas
#         :param data: Datos para predecir
#         :return: TimeSeriesDataFrame con predicciones ensambladas
#         """
#         if not self.results:
#             raise ValueError("Debes llamar a optimize_weights() primero")
        
#         all_preds = []
#         for weight, result in zip(self.weights, self.results):
#             preds = result['predictor'].predict(data)
#             preds['mean'] = preds['mean'] * weight
#             all_preds.append(preds[['mean']])
        
#         # Combinar predicciones
#         ensemble_pred = sum(all_preds)
#         ensemble_pred['0.1'] = all_preds[0]['0.1']  # Mantener intervalos del primer predictor
#         ensemble_pred['0.9'] = all_preds[0]['0.9']
        
#         return ensemble_pred

# # ---------------------------------------------------------------
# # EJEMPLO DE USO COMPLETO
# # ---------------------------------------------------------------

# def main():
#     # 1. Cargar y preparar datos (usando tu código existente)
#     df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
#     dfg = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()
#     dfg['timestamp'] = pd.to_datetime(dfg['periodo'].astype(str), format='%Y%m')
#     dfg.rename(columns={'tn': 'target', 'product_id': 'item_id'}, inplace=True)
    
#     # Filtrar productos
#     productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
#     dfg = dfg[dfg['item_id'].isin(productos_ok['product_id'].unique())]
    
#     # Convertir a TimeSeriesDataFrame
#     data = TimeSeriesDataFrame.from_data_frame(
#         dfg,
#         id_column="item_id",
#         timestamp_column="timestamp"
#     )

#     # 2. Configurar el ensamblador
#     ensemble = ErrorDrivenEnsemble(val_window_size=24)
    
#     # Agregar diferentes configuraciones de modelos
#     ensemble.add_predictor({
#         'name': 'fast_training',
#         'train_kwargs': {
#             'presets': 'fast_training',
#             'num_val_windows': 3
#         }
#     })
    
#     ensemble.add_predictor({
#         'name': 'deepar_12w',
#         'train_kwargs': {
#             'num_val_windows': 12,
#             'hyperparameters': {
#                 'DeepAR': {
#                     'epochs': 50,
#                     'num_cells': 64,
#                     'context_length': 24
#                 }
#             }
#         }
#     })
    
#     ensemble.add_predictor({
#         'name': 'multi_window',
#         'train_kwargs': {
#             'num_val_windows': 6,
#             'hyperparameters': {
#                 'SimpleFeedForward': {'epochs': 30},
#                 'ARIMA': {},
#                 'ETS': {}
#             }
#         }
#     })

#     # 3. Optimizar pesos
#     optimization_result = ensemble.optimize_weights(data)
#     print("\nResultado de optimización:")
#     print(pd.DataFrame(optimization_result))

#     # 4. Generar predicciones finales
#     final_predictions = ensemble.predict(data)
    
#     # Filtrar para febrero 2020 (como en tu ejemplo)
#     preds_202002 = final_predictions.reset_index()
#     preds_202002 = preds_202002[preds_202002['timestamp'] == '2020-02-29']
#     final_result = preds_202002[['item_id', 'mean']].rename(columns={
#         'item_id': 'product_id',
#         'mean': 'pred_ensemble'
#     })
    
#     # Guardar resultados
#     output_path = "./outputs/predicciones_exp_06v3_autogluon.csv"
#     final_result.to_csv(output_path, index=False)
#     print(f"\nPredicciones ensambladas guardadas en {output_path}")



In [7]:
# import numpy as np
# import pandas as pd
# from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
# from sklearn.metrics import mean_absolute_error
# from typing import List, Dict
# from itertools import combinations
# import matplotlib.pyplot as plt

# class ErrorDrivenEnsemble:
#     def __init__(self, val_window_size: int = 24):
#         """
#         Inicializa el ensamblador con el tamaño de la ventana de validación
#         :param val_window_size: Número de períodos a usar para validación (por defecto 24 meses)
#         """
#         self.val_window_size = val_window_size
#         self.predictors = []
#         self.weights = []
#         self.best_combination = None
#         self.results = None

#     def add_predictor(self, predictor_config: Dict):
#         """
#         Agrega una configuración de predictor al ensamblaje
#         """
#         self.predictors.append(predictor_config)

#     def _prepare_validation_data(self, data: TimeSeriesDataFrame):
#         """Prepara los datos de entrenamiento y validación con validaciones"""
#         if len(data) < self.val_window_size:
#             raise ValueError(f"No hay suficientes datos. Necesitas al menos {self.val_window_size} observaciones")
            
#         train_data = data.iloc[:-self.val_window_size]
#         val_data = data.iloc[-self.val_window_size:]
        
#         if val_data['target'].isnull().any():
#             raise ValueError("El conjunto de validación contiene valores NaN en el target")
            
#         return train_data, val_data

#     def _train_predictor(self, train_data: TimeSeriesDataFrame, config: Dict):
#         """Entrena un predictor individual con manejo de errores"""
#         try:
#             predictor = TimeSeriesPredictor(
#                 target='target',
#                 prediction_length=1,  # Cambiado a 1 para predecir mes+2
#                 freq="M",
#                 eval_metric="MAPE",
#                 quantile_levels=[0.1, 0.5, 0.9]
#             )
            
#             predictor.fit(
#                 train_data=train_data,
#                 **config['train_kwargs']
#             )
#             return predictor
#         except Exception as e:
#             print(f"Error durante entrenamiento: {str(e)}")
#             return None

#     def _evaluate_predictor(self, predictor, val_data: TimeSeriesDataFrame):
#         """Versión robusta del evaluador"""
#         if predictor is None:
#             return float('inf'), None
            
#         try:
#             preds = predictor.predict(val_data)
            
#             if len(preds) == 0:
#                 print("⚠️ El predictor no generó predicciones")
#                 return float('inf'), None
                
#             # Asegurar alineación de índices
#             val_targets = val_data['target']
#             aligned_preds = preds['mean'].reindex(val_targets.index)
            
#             if aligned_preds.isnull().any():
#                 print("⚠️ Algunas predicciones no están disponibles para evaluación")
#                 return float('inf'), preds
                
#             mae = mean_absolute_error(val_targets, aligned_preds)
#             return mae, preds
            
#         except Exception as e:
#             print(f"Error durante evaluación: {str(e)}")
#             return float('inf'), None

#     def optimize_weights(self, data: TimeSeriesDataFrame, find_best_subset: bool = True, max_models: int = 3):
#         """Versión mejorada con manejo robusto de errores"""
#         train_data, val_data = self._prepare_validation_data(data)
#         results = []
        
#         for config in self.predictors:
#             print(f"\nEntrenando {config['name']}...")
#             predictor = self._train_predictor(train_data, config)
#             mae, preds = self._evaluate_predictor(predictor, val_data)
            
#             if preds is not None:
#                 results.append({
#                     'name': config['name'],
#                     'predictor': predictor,
#                     'mae': mae,
#                     'val_predictions': preds
#                 })
#                 print(f"{config['name']} - MAE: {mae:.2f}")
        
#         if not results:
#             raise ValueError("Ningún predictor se entrenó exitosamente")
        
#         self.results = results
        
#         if find_best_subset and len(results) > 1:
#             self.best_combination = self.find_best_subset(max_models)
#             selected_indices = [i for i, r in enumerate(results) if r['name'] in self.best_combination['models']]
#             self.results = [results[i] for i in selected_indices]
#             self.weights = self.best_combination['weights']
#         else:
#             maes = np.array([r['mae'] for r in results])
#             self.weights = (1 / maes) / (1 / maes).sum()
#             self.best_combination = {
#                 'names': [r['name'] for r in results],
#                 'weights': self.weights.tolist(),
#                 'mean_mae': (maes * self.weights).sum(),
#                 'best_mae': maes.min()
#             }
        
#         return self.best_combination

#     def find_best_subset(self, max_models=3):
#         """Encuentra la mejor combinación de modelos"""
#         model_names = [r['name'] for r in self.results]
#         best_score = float('inf')
#         best_combo = None
        
#         for k in range(1, min(max_models, len(model_names)) + 1):
#             for combo in combinations(range(len(model_names)), k):
#                 maes = [self.results[i]['mae'] for i in combo]
#                 weights = (1 / np.array(maes)) / (1 / np.array(maes)).sum()
#                 weighted_mae = (np.array(maes) * weights).sum()
                
#                 if weighted_mae < best_score:
#                     best_score = weighted_mae
#                     best_combo = {
#                         'models': [model_names[i] for i in combo],
#                         'weights': weights.tolist(),
#                         'mae': weighted_mae
#                     }
        
#         print(f"\nMejor combinación: {best_combo['models']}")
#         print(f"Pesos: {best_combo['weights']}")
#         print(f"MAE ponderado: {best_combo['mae']:.4f}")
#         return best_combo

#     def plot_ensemble_results(self):
#         """Visualiza resultados"""
#         fig, ax = plt.subplots(figsize=(10, 6))
#         ax.bar([r['name'] for r in self.results], [r['mae'] for r in self.results], label='MAE Individual')
#         ax.axhline(self.best_combination['mae'], color='r', linestyle='--', label='MAE Ensamblado')
#         ax.set_title('Comparación de Modelos')
#         ax.set_ylabel('MAE')
#         ax.legend()
#         plt.xticks(rotation=45)
#         plt.tight_layout()
#         plt.show()

#     def predict(self, data: TimeSeriesDataFrame):
#         """Genera predicciones ensambladas"""
#         all_preds = []
#         for weight, result in zip(self.weights, self.results):
#             preds = result['predictor'].predict(data)
#             preds['mean'] = preds['mean'] * weight
#             all_preds.append(preds[['mean']])
        
#         ensemble_pred = sum(all_preds)
#         ensemble_pred['0.1'] = all_preds[0]['0.1']
#         ensemble_pred['0.9'] = all_preds[0]['0.9']
#         return ensemble_pred

# # Ejemplo de uso
# def main():
#     # 1. Cargar y preparar datos
#     df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
#     df['timestamp'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')
#     df = df.rename(columns={'product_id': 'item_id'}).dropna(subset=['target'])
    
#     # Convertir a TimeSeriesDataFrame
#     data = TimeSeriesDataFrame.from_data_frame(
#         df[['item_id', 'timestamp', 'target']],
#         id_column="item_id",
#         timestamp_column="timestamp"
#     )

#     # 2. Configurar ensamblador
#     ensemble = ErrorDrivenEnsemble(val_window_size=3)
    
#     # Configuraciones de modelos mejoradas
#     ensemble.add_predictor({
#         'name': 'deepar',
#         'train_kwargs': {
#             'hyperparameters': {
#                 'DeepAR': {
#                     'epochs': 50,
#                     'num_cells': 64,
#                     'context_length': 12
#                 }
#             },
#             'num_val_windows': 5
#         }
#     })
    
#     ensemble.add_predictor({
#         'name': 'feedforward',
#         'train_kwargs': {
#             'hyperparameters': {
#                 'SimpleFeedForward': {
#                     'epochs': 30,
#                     'num_hidden_dimensions': [64]
#                 }
#             }
#         }
#     })

#     # 3. Optimizar y evaluar
#     optimization_result = ensemble.optimize_weights(data)
#     ensemble.plot_ensemble_results()
    
#     # 4. Generar y guardar predicciones
#     predictions = ensemble.predict(data)
#     # predictions.reset_index().to_csv("./outputs/ensemble_predictions.csv", index=False)

# if __name__ == "__main__":
#     main()

In [10]:
import numpy as np
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

class SimpleErrorDrivenEnsemble:
    def __init__(self, prediction_length=2, val_window_size=24):
        self.prediction_length = prediction_length
        self.val_window_size = val_window_size
        self.predictors = {}              # {model_name: hyperparameters}
        self.model_errors = {}            # {model_name: mae}
        self.model_weights = {}           # {model_name: weight}
        self.trained_predictors = {}      # {model_name: TimeSeriesPredictor}

    def prepare_data(self, df, id_col='product_id', time_col='periodo', target_col='tn'):
        df['timestamp'] = pd.to_datetime(df[time_col].astype(str), format='%Y%m')
        df = df.rename(columns={id_col: 'item_id'}).dropna(subset=['target'])
        df = df.groupby('item_id').filter(lambda x: x['target'].notna().sum() >= (self.val_window_size + 12))
        return TimeSeriesDataFrame.from_data_frame(
            df[['item_id', 'timestamp', 'target']],
            id_column='item_id',
            timestamp_column='timestamp'
        )

    def add_model(self, name, hyperparameters):
        self.predictors[name] = hyperparameters

    def train_and_evaluate(self, data: TimeSeriesDataFrame):
        train_data, val_data = data.train_test_split(self.val_window_size)

        if len(val_data) == 0:
            raise ValueError("El conjunto de validación quedó vacío. Revisá que haya suficientes datos por serie.")

        for name, hyperparams in self.predictors.items():
            print(f"Entrenando modelo: {name}")
            predictor = TimeSeriesPredictor(
                target='target',
                prediction_length=self.prediction_length,
                freq='M',
                eval_metric='MAPE'
            )
            predictor.fit(train_data, hyperparameters={name: hyperparams}, verbosity=0)
            self.trained_predictors[name] = predictor

            preds = predictor.predict(val_data)

            y_true = val_data['target']
            y_pred = preds['mean'].reindex(y_true.index)

            if len(y_pred) != len(y_true):
                raise ValueError(f"Desajuste entre y_true ({len(y_true)}) y y_pred ({len(y_pred)}).")

            mae = mean_absolute_error(y_true, y_pred)
            self.model_errors[name] = mae

        self._compute_weights()


    def _compute_weights(self):
        errors = pd.Series(self.model_errors)
        weights = (1 / errors) / (1 / errors).sum()
        self.model_weights = weights.to_dict()
        print("\nPesos (calculados en base a error inverso):")
        print(self.model_weights)

    def predict(self, data: TimeSeriesDataFrame):
        ensemble = None
        for name, predictor in self.trained_predictors.items():
            preds = predictor.predict(data)['mean']
            weight = self.model_weights.get(name, 0)
            if ensemble is None:
                ensemble = preds * weight
            else:
                ensemble += preds * weight
        return ensemble.reset_index(name='ensemble_pred')

    def plot_model_errors(self):
        pd.Series(self.model_errors).sort_values().plot(kind='barh', title='MAE por modelo')
        plt.xlabel('MAE')
        plt.tight_layout()
        plt.show()


In [13]:
# 1. Cargar tus datos originales
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv")

# 2. Inicializar ensemble
ensemble = SimpleErrorDrivenEnsemble(prediction_length=1, val_window_size=3)

# 3. Preparar datos
ts_data = ensemble.prepare_data(df)

# 4. Agregar modelos a usar
ensemble.add_model('DeepAR', {'epochs': 30, 'context_length': 12})
ensemble.add_model('SimpleFeedForward', {'epochs': 40, 'num_hidden_dimensions': [64]})

# 5. Entrenar y evaluar
ensemble.train_and_evaluate(ts_data)

# 6. Visualizar errores
ensemble.plot_model_errors()

# 7. Predecir sobre los datos
ensemble_preds = ensemble.predict(ts_data)
print(ensemble_preds.head())


Frequency 'M' stored as 'ME'


Entrenando modelo: DeepAR


	Parameter 'epochs' cannot be specified when 'max_epochs' is also specified.
Trainer has no fit models that can predict.


ValueError: Trainer has no fit models that can predict.